In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from numpy import random
import nltk
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup


from sklearn.naive_bayes import MultinomialNB
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix, classification_report
import matplotlib.pyplot as plt





In [3]:
filename_f = "c:/users/chung/desktop/springboard_stuff/trials/model_3/depression/dep_forum_clean.csv"
A = pd.read_csv(filename_f)
filename_2_f = "c:/users/chung/desktop/springboard_stuff/trials/model_3/ptsd/ptsd_forum_clean.csv"
B = pd.read_csv(filename_2_f)
filename_3_f = "c:/users/chung/desktop/springboard_stuff/trials/model_3/cptsd/cptsd_forum_clean.csv"
C = pd.read_csv(filename_3_f)
filename_4_f = "c:/users/chung/desktop/springboard_stuff/trials/model_3/bpd/bpd_forum_clean.csv"
D = pd.read_csv(filename_4_f)
filename_5_f = "c:/users/chung/desktop/springboard_stuff/trials/model_3/bipolar/bipolar_forum_clean.csv"
E = pd.read_csv(filename_5_f)
filename_6_f = "c:/users/chung/desktop/springboard_stuff/trials/model_3/dissociation/diss_forum_clean.csv"
F = pd.read_csv(filename_6_f)

A_ = A.dropna()
B_ = B.dropna()
C_ = C.dropna()
D_ = D.dropna()
E_ = E.dropna()
F_ = F.dropna()

master = A_.append([B_,C_,D_,E_,F_])
master_df = pd.DataFrame(master)


'''assigning labels and storing in variable to prepare for holdout creation'''
df_dep = master_df.loc[master_df['label'] == 0] 
df_ptsd = master_df.loc[master_df['label'] == 1]
df_cptsd = master_df.loc[master_df['label'] == 2] 
df_bpd = master_df.loc[master_df['label'] == 3]
df_bipolar = master_df.loc[master_df['label'] == 4] 
df_diss = master_df.loc[master_df['label'] == 5]


'''creating holdouts'''
df_dep_holdout = df_dep.iloc[:100]
df_ptsd_holdout = df_ptsd.iloc[:100]
df_cptsd_holdout = df_cptsd.iloc[:100]
df_bpd_holdout = df_bpd.iloc[:100]
df_bipolar_holdout = df_bipolar.iloc[:100]
df_diss_holdout = df_diss.iloc[:100]


'''re-defining variables with holdout documents ommitted'''
df_dep = df_dep.iloc[100:]
df_ptsd = df_ptsd.iloc[100:]
df_cptsd = df_cptsd.iloc[100:]
df_bpd = df_bpd.iloc[100:]
df_bipolar = df_bipolar.iloc[100:]
df_diss = df_diss.iloc[100:]


'''concat the pieces back together'''
dfnew = pd.concat([df_dep, df_ptsd, df_cptsd, df_bpd, df_bipolar, df_diss])
df_holdout = pd.concat([df_dep_holdout, df_ptsd_holdout, df_cptsd_holdout, df_bpd_holdout, df_bipolar_holdout, df_diss_holdout])

dfnew.head(10)

,text,label
100,"['anyone', 'else', 'intrusive', 'thought', 'co...",0
101,"['agony', 'started', 'came', 'antidepressant',...",0
102,"['two', 'week', 'taking', 'sertraline', 'mood'...",0
103,"['hi', 'honestly', 'know', 'wrong', 'going', '...",0
104,"['hi', 'one', 'like', 'birthday', 'past', 'neg...",0
105,"['able', 'control', '2', 'day', 'getting', 'mu...",0
106,"['diagnosed', 'depression', 'anxiety', 'dec', ...",0
107,"['depressed', 'since', '16', 'recently', 'diag...",0
108,"['remember', 'exactly', 'last', 'time', 'last'...",0
109,"['unfortunate', 'circumstance', 'product', 'mi...",0


In [7]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = str(text)
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text
    
dfnew['text'] = dfnew['text'].apply(clean_text)
df_holdout['text'] = df_holdout['text'].apply(clean_text)

dfnew.to_csv('master_f.csv', index=False)
df_holdout.to_csv('master_f_holdout.csv', index=False)

print(dfnew.head())
print(df_holdout.head())

X = dfnew['text'].astype(str).dropna()
y = dfnew['label']

vectorizer = TfidfVectorizer(
    stop_words='english', 
    min_df = 5,
    ngram_range = (1,2),
    use_idf = True,
    max_df = 0.5, 
    smooth_idf=True)

X = vectorizer.fit_transform(X).toarray()

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2, 
    shuffle=True,
    random_state=42
)

                                                  text  label
100  anyone else intrusive thought convince situati...      0
101  agony started came antidepressant several year...      0
102  two week taking sertraline mood improved bit f...      0
103  hi honestly know wrong going honest see think ...      0
104  hi one like birthday past negative thing big e...      0
                                                text  label
0                             saw today wanted share      0
1  understand situation permanent one sure make b...      0
2  suicidal thought really common something happe...      0
3  want share experience read lot topic time peop...      0
4  article talk depressive wear smile get world h...      0


In [4]:
model_dict={
    'MNB': MultinomialNB(),
    'SGD': SGDClassifier(),
    'LOG': LogisticRegression(),
    'RF': RandomForestClassifier(),
    'ADA': AdaBoostClassifier(),
    'GNB': GaussianNB(),
    'DT': DecisionTreeClassifier(),
    'KNN': KNeighborsClassifier(),
    'DUM': DummyClassifier(),
}

def evaluate(model_dict, X_train, X_test, y_train, y_test):
    
    for k, v in model_dict.items():
        model = v
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

       
        print(k)
        print(classification_report(y_test, y_pred))
        
evaluate(model_dict, X_train, X_test, y_train, y_test)

MNB
              precision    recall  f1-score   support

           0       0.50      0.74      0.60       486
           1       1.00      0.01      0.01       171
           2       0.94      0.24      0.38       253
           3       0.44      0.65      0.52       453
           4       0.57      0.70      0.63       471
           5       1.00      0.02      0.04       192

    accuracy                           0.52      2026
   macro avg       0.74      0.39      0.36      2026
weighted avg       0.65      0.52      0.46      2026

SGD
              precision    recall  f1-score   support

           0       0.63      0.74      0.68       486
           1       0.73      0.58      0.65       171
           2       0.74      0.81      0.77       253
           3       0.65      0.68      0.66       453
           4       0.79      0.65      0.71       471
           5       0.62      0.57      0.59       192

    accuracy                           0.68      2026
   macro avg   

C:\Users\chung\anaconda3\envs\TestEnv\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LOG
              precision    recall  f1-score   support

           0       0.61      0.74      0.67       486
           1       0.80      0.46      0.58       171
           2       0.81      0.74      0.77       253
           3       0.57      0.72      0.64       453
           4       0.71      0.67      0.69       471
           5       0.75      0.40      0.52       192

    accuracy                           0.66      2026
   macro avg       0.71      0.62      0.65      2026
weighted avg       0.68      0.66      0.66      2026

RF
              precision    recall  f1-score   support

           0       0.58      0.75      0.66       486
           1       0.73      0.52      0.61       171
           2       0.77      0.68      0.72       253
           3       0.55      0.65      0.59       453
           4       0.73      0.69      0.71       471
           5       0.64      0.24      0.35       192

    accuracy                           0.64      2026
   macro avg    

C:\Users\chung\anaconda3\envs\TestEnv\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\chung\anaconda3\envs\TestEnv\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\chung\anaconda3\envs\TestEnv\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
